In [6]:
import BioSimSpace as bss
import os

In [7]:
def change_integrator(system: bss._SireWrappers._system.System,protocol: bss.Protocol._minimisation.Minimisation,
                      work_dir: str, process_name: str) -> None:
    """
    Change the MD integrator in the minimisation process
    @param: system
    @param: protocol
    @param: work_dir
    @param: process_name
    @return: None
    """
    try:
        process = bss.Process.Gromacs(system, protocol, name=process_name, work_dir=work_dir)
    except RuntimeError:
        with open(f"{work_dir}/{process_name}.mdp", "r") as mdp:
            lines = mdp.read()
        new_lines = lines.replace("integrator = cg", "integrator = steep")
        with open(f"{work_dir}/{process_name}.mdp", "w") as mdp:
            mdp.write(new_lines)


In [2]:
minimisation_steps = 2000
short_nvt_runtime = 5 # ps
nvt_runtime = 50 # ps
npt_runtime = 200 # ps 


In [9]:
n_ligands = 2
for i in range(n_ligands):
    ligand_datafile = open("ligands.dat", "r")
    ligand_lines = ligand_datafile.readlines()
    ligand_name = ligand_lines[i].rstrip()
    os.system(f"mkdir -p ../runs/equilibration/ligands/{ligand_name}")
    ligand_work_dir = f"../runs/equilibration/ligands/{ligand_name}"
    solvated_ligand = bss.IO.readMolecules([f"../inputs/ligands/{ligand_name}_solv.prm7",
                                            f"../inputs/ligands/{ligand_name}_solv.rst7"])
    ligand_minimisation_protocol = bss.Protocol.Minimisation(steps=minimisation_steps)
    change_integrator(solvated_ligand, ligand_minimisation_protocol, ligand_work_dir, "min")
    ligand_minimisation_process = bss.Process.Gromacs(solvated_ligand,
                                                      ligand_minimisation_protocol,
                                                      name="min",
                                                      work_dir=ligand_work_dir)
    gmx_grompp = f"gmx grompp -f {ligand_work_dir}/min.mdp -c {ligand_work_dir}/min.gro -p {ligand_work_dir}/min.top -o {ligand_work_dir}/min.tpr"
    os.system(gmx_grompp)
    gmx_mdrun = f"gmx mdrun -v -deffnm {ligand_work_dir}/min -nt 1 -nb gpu"
    os.system(gmx_mdrun)

Setting the LD random seed to -272498755

Generated 136 of the 136 non-bonded parameter combinations

Generated 136 of the 136 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'MOL'

Excluding 3 bonded neighbours molecule type 'SOL'

Excluding 3 bonded neighbours molecule type 'NA'
Analysing residue names:
There are:     1      Other residues
There are:   640      Water residues
There are:     1        Ion residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

The largest distance between excluded atoms is 0.408 nm
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 24x24x24, spacing 0.113 0.113 0.113

Estimate for the relative computational load of the PME mesh part: 0.30

This run will generate roughly 0 Mb of data
Setting the LD random seed to -1174448129

Generated 136 of the 136 non-bonded parameter combinations